<a href="https://colab.research.google.com/github/SharadhiVK/code/blob/main/Benign%20Capital%20Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import numpy as np


In [8]:
api_key = "Z89DVHRA61V016LD"
symbol = "IBM"
interval = "5min"

url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={api_key}"

response = requests.get(url)
data = response.json()

# Extract the time series data
time_series_data = data["Time Series (5min)"]

# Convert the data to a DataFrame
df = pd.DataFrame.from_dict(time_series_data, orient="index")
df.index = pd.to_datetime(df.index)
df.columns = ["open", "high", "low", "close", "volume"]
df = df.astype(float)


In [12]:
# Calculate Super Trend
def calculate_super_trend(df, atr_period, multiplier):
    df["basic_upper"] = (df["high"] + df["low"]) / 2 + multiplier * df["atr"]
    df["basic_lower"] = (df["high"] + df["low"]) / 2 - multiplier * df["atr"]
    df["final_upper"] = np.nan  # Initialize with NaN for the first row
    df["final_lower"] = np.nan  # Initialize with NaN for the first row

    for i in range(1, len(df)):
        if df.iloc[i - 1]["close"] <= df.iloc[i - 1]["final_upper"]:
            df.at[df.index[i], "final_upper"] = min(df.iloc[i - 1]["basic_upper"], df.iloc[i - 1]["final_upper"])
        else:
            df.at[df.index[i], "final_upper"] = df.iloc[i - 1]["basic_upper"]

        if df.iloc[i - 1]["close"] >= df.iloc[i - 1]["final_lower"]:
            df.at[df.index[i], "final_lower"] = max(df.iloc[i - 1]["basic_lower"], df.iloc[i - 1]["final_lower"])
        else:
            df.at[df.index[i], "final_lower"] = df.iloc[i - 1]["basic_lower"]

calculate_super_trend(df, atr_period=7, multiplier=3)

# Display the DataFrame with Super Trend values
print(df[["close", "final_upper", "final_lower"]])


                       close  final_upper  final_lower
2023-09-06 19:55:00  147.820          NaN          NaN
2023-09-06 19:50:00  147.900          NaN          NaN
2023-09-06 19:45:00  147.840          NaN          NaN
2023-09-06 19:40:00  147.850          NaN          NaN
2023-09-06 19:25:00  147.820          NaN          NaN
...                      ...          ...          ...
2023-09-06 10:15:00  148.090   148.353571   147.631143
2023-09-06 10:10:00  148.100   148.353571   147.631143
2023-09-06 10:05:00  147.990   148.353571   147.631143
2023-09-06 10:00:00  147.865   148.353571   147.631143
2023-09-06 09:55:00  147.990   148.353571   147.631143

[100 rows x 3 columns]
